# Test

## Import packages

In [ ]:
import tensorflow_datasets as tfds

from tensorflow.lite.python.schema_py_generated import Model

import timm
import torch
import os
import shutil
import pandas as pd
from timm.data import create_dataset


In [ ]:
os.getcwd()

### Inspecting timm

In [ ]:
len(timm.list_models())

In [ ]:
len(timm.list_models(pretrained=True))

#### Load data

In [ ]:
# Construct a tf.data.Dataset
ds = tfds.load('mnist', split='train', as_supervised=True, shuffle_files=True)

In [ ]:
os.chdir(r'C:\\Users\\marti\\Desktop\\Škola\\Diplomova prace\\Imagenette\\Imagenette2')

In [ ]:

noise_level = '5' # '5' or '50'
outdir = 'noisy' + noise_level
df = pd.read_csv('noisy_imagenette.csv', sep=',')
for inpath, label, is_val in zip(df['path'], df['noisy_labels_' + noise_level], df['is_valid']):
    if is_val:
        subdir = 'val'
    else:
        subdir = 'train'
    outpath = os.path.join(outdir, subdir, label)
    os.makedirs(outpath, exist_ok=True)
    shutil.copy2(inpath, outpath)

In [ ]:
from datasets import load_dataset

mnist = load_dataset("ylecun/mnist")

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
os.chdir(r'C:\\Users\\marti\\Desktop\\Škola\\Diplomova prace\\CIFAR')

In [ ]:
cifar = create_dataset('torch/cifar10', 'cifar10', download=True, split='train')

# Model

In [ ]:
timm.list_models('efficientnet*', pretrained=True)

In [ ]:
model = timm.create_model('resnet50d', pretrained=True, num_classes=10)

In [ ]:
model

In [ ]:
# %%writefile train.py

import argparse
from pathlib import Path

import timm
import timm.data
import timm.loss
import timm.optim
import timm.utils
import torch
import torchmetrics
from timm.scheduler import CosineLRScheduler

from pytorch_accelerated.callbacks import SaveBestModelCallback
from pytorch_accelerated.trainer import Trainer, DEFAULT_CALLBACKS


def create_datasets(image_size, data_mean, data_std, train_path, val_path):
    train_transforms = timm.data.create_transform(
        input_size=image_size,
        is_training=True,
        mean=data_mean,
        std=data_std,
        auto_augment="rand-m7-mstd0.5-inc1",
    )

    eval_transforms = timm.data.create_transform(
        input_size=image_size, mean=data_mean, std=data_std
    )

    train_dataset = timm.data.dataset.ImageDataset(
        train_path, transform=train_transforms
    )
    eval_dataset = timm.data.dataset.ImageDataset(val_path, transform=eval_transforms)

    return train_dataset, eval_dataset


class TimmMixupTrainer(Trainer):
    def __init__(self, eval_loss_fn, mixup_args, num_classes, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_loss_fn = eval_loss_fn
        self.num_updates = None
        self.mixup_fn = timm.data.Mixup(**mixup_args)

        self.accuracy = torchmetrics.Accuracy(num_classes=num_classes)
        self.ema_accuracy = torchmetrics.Accuracy(num_classes=num_classes)
        self.ema_model = None

    def create_scheduler(self):
        return timm.scheduler.CosineLRScheduler(
            self.optimizer,
            t_initial=self.run_config.num_epochs,
            cycle_decay=0.5,
            lr_min=1e-6,
            t_in_epochs=True,
            warmup_t=3,
            warmup_lr_init=1e-4,
            cycle_limit=1,
        )

    def training_run_start(self):
        # Model EMA requires the model without a DDP wrapper and before sync batchnorm conversion
        self.ema_model = timm.utils.ModelEmaV2(
            self._accelerator.unwrap_model(self.model), decay=0.9
        )
        if self.run_config.is_distributed:
            self.model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(self.model)

    def train_epoch_start(self):
        super().train_epoch_start()
        self.num_updates = self.run_history.current_epoch * len(self._train_dataloader)

    def calculate_train_batch_loss(self, batch):
        xb, yb = batch
        mixup_xb, mixup_yb = self.mixup_fn(xb, yb)
        return super().calculate_train_batch_loss((mixup_xb, mixup_yb))

    def train_epoch_end(
        self,
    ):
        self.ema_model.update(self.model)
        self.ema_model.eval()

        if hasattr(self.optimizer, "sync_lookahead"):
            self.optimizer.sync_lookahead()

    def scheduler_step(self):
        self.num_updates += 1
        if self.scheduler is not None:
            self.scheduler.step_update(num_updates=self.num_updates)

    def calculate_eval_batch_loss(self, batch):
        with torch.no_grad():
            xb, yb = batch
            outputs = self.model(xb)
            val_loss = self.eval_loss_fn(outputs, yb)
            self.accuracy.update(outputs.argmax(-1), yb)

            ema_model_preds = self.ema_model.module(xb).argmax(-1)
            self.ema_accuracy.update(ema_model_preds, yb)

        return {"loss": val_loss, "model_outputs": outputs, "batch_size": xb.size(0)}

    def eval_epoch_end(self):
        super().eval_epoch_end()

        if self.scheduler is not None:
            self.scheduler.step(self.run_history.current_epoch + 1)

        self.run_history.update_metric("accuracy", self.accuracy.compute().cpu())
        self.run_history.update_metric(
            "ema_model_accuracy", self.ema_accuracy.compute().cpu()
        )
        self.accuracy.reset()
        self.ema_accuracy.reset()


def main(data_path):

    # Set training arguments, hardcoded here for clarity
    image_size = (224, 224)
    lr = 5e-3
    smoothing = 0.1
    mixup = 0.2
    cutmix = 1.0
    batch_size = 32
    bce_target_thresh = 0.2
    num_epochs = 40

    data_path = Path(data_path)
    train_path = data_path / "train"
    val_path = data_path / "val"
    num_classes = len(list(train_path.iterdir()))

    mixup_args = dict(
        mixup_alpha=mixup,
        cutmix_alpha=cutmix,
        label_smoothing=smoothing,
        num_classes=num_classes,
    )

    # Create model using timm
    model = timm.create_model(
        "resnet50d", pretrained=False, num_classes=num_classes, drop_path_rate=0.05
    )

    # Load data config associated with the model to use in data augmentation pipeline
    data_config = timm.data.resolve_data_config({}, model=model, verbose=True)
    data_mean = data_config["mean"]
    data_std = data_config["std"]

    # Create training and validation datasets
    train_dataset, eval_dataset = create_datasets(
        train_path=train_path,
        val_path=val_path,
        image_size=image_size,
        data_mean=data_mean,
        data_std=data_std,
    )

    # Create optimizer
    optimizer = timm.optim.create_optimizer_v2(
        model, opt="lookahead_AdamW", lr=lr, weight_decay=0.01
    )

    # As we are using Mixup, we can use BCE during training and CE for evaluation
    train_loss_fn = timm.loss.BinaryCrossEntropy(
        target_threshold=bce_target_thresh, smoothing=smoothing
    )
    validate_loss_fn = torch.nn.CrossEntropyLoss()

    # Create trainer and start training
    trainer = TimmMixupTrainer(
        model=model,
        optimizer=optimizer,
        loss_func=train_loss_fn,
        eval_loss_fn=validate_loss_fn,
        mixup_args=mixup_args,
        num_classes=num_classes,
        callbacks=[
            *DEFAULT_CALLBACKS,
            SaveBestModelCallback(watch_metric="accuracy", greater_is_better=True),
        ],
    )

    trainer.train(
        per_device_batch_size=batch_size,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        num_epochs=num_epochs,
        create_scheduler_fn=trainer.create_scheduler,
    )


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Simple example of training script using timm.")
    parser.add_argument("--data_dir", required=True, help="The data folder on disk.")
    args = parser.parse_args()
    main(args.data_dir)

In [ ]:
print(torch.cuda.is_available())

In [ ]:
print(torch.__version__)

In [ ]:
import os
print(os.path.exists('C:\\Users\\marti\\Desktop\\Škola\\Diplomova prace\\Imagenette\imagenette2'))

In [ ]:
import wandb
import random

In [ ]:
wandb.require("core")

In [ ]:
wandb.login()

In [ ]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
print(torch.cuda.memory_summary())

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
print(torch.cuda.list_gpu_processes())

In [ ]:
print(torch.cuda.mem_get_info())

In [ ]:
print(torch.cuda.memory_stats())

In [ ]:
print(torch.cuda.memory_snapshot())

In [ ]:
print(torch.cuda.memory_allocated())

In [ ]:
print(torch.cuda.max_memory_allocated())

In [ ]:
print(torch.cuda.memory_reserved())

In [ ]:
torch.cuda.reset_peak_memory_stats()

In [8]:
print(sweep_config)

{'program': 'train.py', 'method': 'grid', 'parameters': {'initialization': {'values': ['goog', 'custom']}, 'seed': {'values': [1, 2, 3, 4, 5]}}, 'command': ['${program}', './CIFAR/cifar10', '--dataset', 'torch/cifar10', '--model', 'efficientnetv2_s', '--num-classes', '10', '--input-size', '3 32 32', '--batch-size', '128', '--amp', '--no-ddp-bb', '--opt', 'adamw', '--weight-decay', '0.1', '--clip-grad', '1.5', '--sched-on-updates', '--lr', '0.001', '--min-lr', '1e-6', '--epochs', '5', '--warmup-epochs', '1', '--warmup-lr', '1e-6', '--aa', 'v0', '--seed', '${seed}', '--log-wandb', '--project-wandb', 'COMPARE_INIT_SWEEP', '--run-wandb', 'SWEEP_${initialization}_INIT_${seed}', '--model-kwargs', 'initialization=${initialization}']}


In [9]:
import wandb
import yaml
from train import main

# Load the sweeper.yaml configuration from the YAML file
with open('sweep_config.yaml', 'r') as file:
    sweep_config = yaml.safe_load(file)

# Initialize the sweeper.yaml by passing the loaded config and project name
sweep_id = wandb.sweep(sweep=sweep_config)

# Start the sweeper.yaml agent, passing the main function
# The count parameter specifies how many runs you want
wandb.agent(sweep_id)

Create sweep with ID: 82nligu1
Sweep URL: https://wandb.ai/martinhruska_2000/uncategorized/sweeps/82nligu1


Running runs: []
Running runs: []
Running runs: []
Running runs: []
Running runs: []
Running runs: []
Running runs: []
Running runs: []
Running runs: []
Running runs: []
Running runs: []
Agent received command: run
Agent received command: run
Agent received command: run
Agent received command: run
Agent received command: run
Agent received command: run
Agent received command: run
Agent received command: run
Agent received command: run
Agent received command: run
Agent received command: run
Agent starting run with config:
	initialization: goog
	seed: 1
Agent starting run with config:
	initialization: goog
	seed: 1
Agent starting run with config:
	initialization: goog
	seed: 1
Agent starting run with config:
	initialization: goog
	seed: 1
Agent starting run with config:
	initialization: goog
	seed: 1
Agent starting run with config:
	initialization: goog
	seed: 1
Agent starting run with config:
	initialization: goog
	seed: 1
Agent starting run with config:
	initialization: goog
	seed: 1
A

wandb: Agent Starting Run: i9lb4dvw with config:
	initialization: goog
	seed: 1


About to run command: train.py ./CIFAR/cifar10 --dataset torch/cifar10 --model efficientnetv2_s --num-classes 10 --input-size "3 32 32" --batch-size 128 --amp --no-ddp-bb --opt adamw --weight-decay 0.1 --clip-grad 1.5 --sched-on-updates --lr 0.001 --min-lr 1e-06 --epochs 5 --warmup-epochs 1 --warmup-lr 1e-06 --aa v0 --seed --log-wandb --project-wandb COMPARE_INIT_SWEEP --run-wandb SWEEP_${initialization}_INIT_${seed} --model-kwargs initialization=${initialization}
About to run command: train.py ./CIFAR/cifar10 --dataset torch/cifar10 --model efficientnetv2_s --num-classes 10 --input-size "3 32 32" --batch-size 128 --amp --no-ddp-bb --opt adamw --weight-decay 0.1 --clip-grad 1.5 --sched-on-updates --lr 0.001 --min-lr 1e-06 --epochs 5 --warmup-epochs 1 --warmup-lr 1e-06 --aa v0 --seed --log-wandb --project-wandb COMPARE_INIT_SWEEP --run-wandb SWEEP_${initialization}_INIT_${seed} --model-kwargs initialization=${initialization}
About to run command: train.py ./CIFAR/cifar10 --dataset torch

wandb: Agent Starting Run: ik5ajl5w with config:
	initialization: goog
	seed: 2


About to run command: train.py ./CIFAR/cifar10 --dataset torch/cifar10 --model efficientnetv2_s --num-classes 10 --input-size "3 32 32" --batch-size 128 --amp --no-ddp-bb --opt adamw --weight-decay 0.1 --clip-grad 1.5 --sched-on-updates --lr 0.001 --min-lr 1e-06 --epochs 5 --warmup-epochs 1 --warmup-lr 1e-06 --aa v0 --seed --log-wandb --project-wandb COMPARE_INIT_SWEEP --run-wandb SWEEP_${initialization}_INIT_${seed} --model-kwargs initialization=${initialization}
About to run command: train.py ./CIFAR/cifar10 --dataset torch/cifar10 --model efficientnetv2_s --num-classes 10 --input-size "3 32 32" --batch-size 128 --amp --no-ddp-bb --opt adamw --weight-decay 0.1 --clip-grad 1.5 --sched-on-updates --lr 0.001 --min-lr 1e-06 --epochs 5 --warmup-epochs 1 --warmup-lr 1e-06 --aa v0 --seed --log-wandb --project-wandb COMPARE_INIT_SWEEP --run-wandb SWEEP_${initialization}_INIT_${seed} --model-kwargs initialization=${initialization}
About to run command: train.py ./CIFAR/cifar10 --dataset torch

wandb: Agent Starting Run: zump76lu with config:
	initialization: goog
	seed: 3


About to run command: train.py ./CIFAR/cifar10 --dataset torch/cifar10 --model efficientnetv2_s --num-classes 10 --input-size "3 32 32" --batch-size 128 --amp --no-ddp-bb --opt adamw --weight-decay 0.1 --clip-grad 1.5 --sched-on-updates --lr 0.001 --min-lr 1e-06 --epochs 5 --warmup-epochs 1 --warmup-lr 1e-06 --aa v0 --seed --log-wandb --project-wandb COMPARE_INIT_SWEEP --run-wandb SWEEP_${initialization}_INIT_${seed} --model-kwargs initialization=${initialization}
About to run command: train.py ./CIFAR/cifar10 --dataset torch/cifar10 --model efficientnetv2_s --num-classes 10 --input-size "3 32 32" --batch-size 128 --amp --no-ddp-bb --opt adamw --weight-decay 0.1 --clip-grad 1.5 --sched-on-updates --lr 0.001 --min-lr 1e-06 --epochs 5 --warmup-epochs 1 --warmup-lr 1e-06 --aa v0 --seed --log-wandb --project-wandb COMPARE_INIT_SWEEP --run-wandb SWEEP_${initialization}_INIT_${seed} --model-kwargs initialization=${initialization}
About to run command: train.py ./CIFAR/cifar10 --dataset torch

In [8]:
wandb.sweep(sweep=sweep_config)

Create sweep with ID: rgcjpcod
Sweep URL: https://wandb.ai/martinhruska_2000/%3Cyour_project_name%3E/sweeps/rgcjpcod


'rgcjpcod'

In [ ]:
wandb.agent(sweep_id="rgcjpcod", count=1)

wandb: Agent Starting Run: aaa71grd with config:
	initialization: goog
	seed: 1


2024-10-13 23:49:55,840 - wandb.wandb_agent - ERROR - Exception while processing command: {'run_id': 'aaa71grd', 'program': 'train.py', 'type': 'run', 'args': {'initialization': {'value': 'goog'}, 'seed': {'value': 1}}, 'runqueue_item_id': 'UnVuUXVldWVJdGVtOjU5NzQ2MjkzNQ==', 'logs': [], 'run_storage_id': 'UnVuOnYxOmFhYTcxZ3JkOjx5b3VyX3Byb2plY3RfbmFtZT46bWFydGluaHJ1c2thXzIwMDA='}
Traceback (most recent call last):
  File "C:\Users\marti\AppData\Roaming\Python\Python38\site-packages\wandb\wandb_agent.py", line 326, in _process_command
    result = self._command_run(command)
  File "C:\Users\marti\AppData\Roaming\Python\Python38\site-packages\wandb\wandb_agent.py", line 425, in _command_run
    proc = AgentProcess(command=command_list, env=env)
  File "C:\Users\marti\AppData\Roaming\Python\Python38\site-packages\wandb\wandb_agent.py", line 48, in __init__
    self._popen = subprocess.Popen(command, env=env, **kwargs)
  File "C:\Program Files\Python38\lib\subprocess.py", line 858, in __ini

wandb: Agent Starting Run: sc7gwroc with config:
	initialization: goog
	seed: 2


2024-10-13 23:50:01,155 - wandb.wandb_agent - ERROR - Exception while processing command: {'run_id': 'sc7gwroc', 'program': 'train.py', 'type': 'run', 'args': {'initialization': {'value': 'goog'}, 'seed': {'value': 2}}, 'runqueue_item_id': 'UnVuUXVldWVJdGVtOjU5NzQ2MjkzNg==', 'logs': [], 'run_storage_id': 'UnVuOnYxOnNjN2d3cm9jOjx5b3VyX3Byb2plY3RfbmFtZT46bWFydGluaHJ1c2thXzIwMDA='}
Traceback (most recent call last):
  File "C:\Users\marti\AppData\Roaming\Python\Python38\site-packages\wandb\wandb_agent.py", line 326, in _process_command
    result = self._command_run(command)
  File "C:\Users\marti\AppData\Roaming\Python\Python38\site-packages\wandb\wandb_agent.py", line 425, in _command_run
    proc = AgentProcess(command=command_list, env=env)
  File "C:\Users\marti\AppData\Roaming\Python\Python38\site-packages\wandb\wandb_agent.py", line 48, in __init__
    self._popen = subprocess.Popen(command, env=env, **kwargs)
  File "C:\Program Files\Python38\lib\subprocess.py", line 858, in __ini

wandb: Agent Starting Run: znj4y282 with config:
	initialization: goog
	seed: 3


2024-10-13 23:50:06,429 - wandb.wandb_agent - ERROR - Exception while processing command: {'run_id': 'znj4y282', 'program': 'train.py', 'type': 'run', 'args': {'initialization': {'value': 'goog'}, 'seed': {'value': 3}}, 'runqueue_item_id': 'UnVuUXVldWVJdGVtOjU5NzQ2MjkzNw==', 'logs': [], 'run_storage_id': 'UnVuOnYxOnpuajR5MjgyOjx5b3VyX3Byb2plY3RfbmFtZT46bWFydGluaHJ1c2thXzIwMDA='}
Traceback (most recent call last):
  File "C:\Users\marti\AppData\Roaming\Python\Python38\site-packages\wandb\wandb_agent.py", line 326, in _process_command
    result = self._command_run(command)
  File "C:\Users\marti\AppData\Roaming\Python\Python38\site-packages\wandb\wandb_agent.py", line 425, in _command_run
    proc = AgentProcess(command=command_list, env=env)
  File "C:\Users\marti\AppData\Roaming\Python\Python38\site-packages\wandb\wandb_agent.py", line 48, in __init__
    self._popen = subprocess.Popen(command, env=env, **kwargs)
  File "C:\Program Files\Python38\lib\subprocess.py", line 858, in __ini